In [10]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn

import numpy as np
import torch.optim as optim
import torch.utils.data as data
from sklearn.model_selection import train_test_split

In [51]:
dataset = pd.read_csv('../landmark_extraction_mechanism/labeled_dataset.csv')

In [52]:
unlabel = pd.read_csv('../landmark_extraction_mechanism/dataset.csv')

In [53]:
unlabel['label'] = dataset['Label']
dataset = unlabel

### Dataset csv

Each row contains a video (so we can label the data), also the timestamp of the frame where that sequence set of landmarks was captured and 

In [54]:
video = dataset[dataset['video'] == 'video3.mp4']

In [56]:
video

,video,group,frame,landmarks,label
0,video3.mp4,1,5,[ 0 0 288.62 254....,good
1,video3.mp4,1,11,[ 0 0 287.18 255....,good
2,video3.mp4,1,17,[ 0 0 284.14 255....,good
3,video3.mp4,1,23,[ 0 0 284.85 255...,good
4,video3.mp4,1,29,[ 0 0 285.81 255....,good
5,video3.mp4,1,35,[ 0 0 286.57 255....,good
6,video3.mp4,1,41,[ 0 0 287.48 255....,good
7,video3.mp4,1,47,[ 0 0 287.85 2...,good
8,video3.mp4,1,53,[ 0 0 288.13 254....,good
9,video3.mp4,1,59,[ 0 0 289.34 254....,good


In [9]:

class AirModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.linear(out)  # Take the last time step's output
        return out


In [10]:
# Define your training data

# Create an instance of the AirModel
model = AirModel()

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Set the number of training epochs
num_epochs = 100000

# Train the model
for epoch in range(num_epochs):
    model.train()
    
    # Forward pass
    outputs = model(train_X)
    loss = criterion(outputs, train_y)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print training loss for every few epochs
    if (epoch+1) % int(num_epochs*0.25) == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save the trained model
torch.save(model.state_dict(), 'air_model.pth')

Epoch [25000/100000], Loss: 15448.4248
Epoch [50000/100000], Loss: 15364.0752
Epoch [75000/100000], Loss: 15323.7959
Epoch [100000/100000], Loss: 15295.6182


In [11]:

# Assuming 'test_X' and 'test_y' are your test data NumPy arrays
test_X = torch.tensor(test_X, dtype=torch.float32)
test_y = torch.tensor(test_y, dtype=torch.float32)

# Set the model to evaluation mode
model.eval()

# Make predictions on the test set
with torch.no_grad():
    predicted_y = model(test_X)

# Calculate the accuracy
absolute_diff = torch.abs(predicted_y - test_y)
accuracy = 1.0 - torch.mean(absolute_diff / test_y)

print(f"Accuracy: {accuracy.item()*100:.2f}%")


Accuracy: 50.56%
